In [ ]:
!pip install -q zarr dask[distributed] intake xarray fsspec aiohttp regionmask --upgrade
!pip install -q git+https://github.com/carbonplan/cmip6-downscaling.git@1.0

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from dask.diagnostics import ProgressBar
base_url = 'https://esiptutorial.blob.core.windows.net/eraswe'

In [ ]:
years = np.arange(1985, 2015)
swe_files = [f'{base_url}/era5_raw_swe/era5_raw_swe_{y}.zarr' for y in years]
ds = xr.open_mfdataset(swe_files, engine='zarr')

mask = xr.open_dataset(f'{base_url}/mask_10k_household.zarr', engine='zarr')
mask = mask.rename({'sd': 'mask'})

terrain = xr.open_dataset(f'{base_url}/processed_slope_aspect_elevation.zarr', engine='zarr')
terrain = terrain.rename({'lat': 'latitude', 'lon': 'longitude'})

ds = xr.merge([ds, mask, terrain])

print(f"This data is {ds.nbytes / 1e9} GB!")
print("--------------------------------------")
ds

In [ ]:
rainier_lat, rainier_lon = 46.85, 121.76
with ProgressBar():
    ds['sd'].sel(
        latitude=rainier_lat, 
        longitude=rainier_lon, 
        method='nearest'
    ).plot()

In [ ]:
year = '2005'
with ProgressBar():
    swe_year = ds['sd'].sel(time=year)
    (swe_year.groupby(swe_year.time.dt.season)
             .mean()
             .plot(col='season', col_wrap=2, vmin=0, vmax=0.1))